<a href="https://colab.research.google.com/github/evefine/test/blob/main/epsilon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from scipy.stats import linregress

import pandas as pd
import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting
from bokeh.models import HoverTool, WheelZoomTool, PanTool, BoxZoomTool, ResetTool, TapTool, SaveTool
import io
bokeh.io.output_notebook()

# Eve did this
from google.colab import files
 

g_colab = False

In [3]:
g_colab = False
g_colab = input('Are you using google colab(Y/N)? ')

if g_colab.upper() == 'Y':
    g_colab = True
    print('select ALL FILES NEEDED (hold command and click all files needed)')
    uploaded = files.upload()
else:
    g_colab = False

Are you using google colab(Y/N)? Y
select ALL FILES NEEDED


Saving EJF_10_50.csv to EJF_10_50.csv
Saving experiment_conditions_11_6.csv to experiment_conditions_11_6.csv
Saving experiment_vals_11_6.csv to experiment_vals_11_6.csv


**Please be sure to upload all files that you will need throughout the notebook. I am pasting below what my code
spit out at me after I uploaded what I needed (and afterwards successfully ran the notebook in its entirety):**


EJF_10_50.csv(text/csv) - 7096 bytes, last modified: 12/18/2022 - 100% done

experiment_conditions_11_6.csv(text/csv) - 359 bytes, last modified: 12/6/2022 - 100% done

experiment_vals_11_6.csv(text/csv) - 162754 bytes, last modified: 12/7/2022 - 100% done

Saving EJF_10_50.csv to EJF_10_50.csv

Saving experiment_conditions_11_6.csv to experiment_conditions_11_6.csv

Saving experiment_vals_11_6.csv to experiment_vals_11_6.csv

In [4]:
fname_un = 'EJF_10_50.csv'
if g_colab:
    df = pd.read_csv(io.BytesIO(uploaded[fname_un]))
else:
    df = pd.read_csv(fname_un)

df.head()

,Wavel.,0,50,10
0,350,0.1352,0.2044,0.1365
1,352,0.1325,0.2089,0.1354
2,354,0.1283,0.2148,0.1334
3,356,0.1243,0.2203,0.1319
4,358,0.1195,0.2249,0.1291


These data are the absorbance spectra of
iNicSnFR12 at the respective concentrations
of 0, 50, and 10 $\mu$M of iNicSnFR12 ***without*** 
any nicotine present. I realize that
the data is in the wrong order, but this
is not an issue. The column name corresponds
to the correct set of data.

This means that all protein present is in the apo,
or unbound state. My goal with this data is to
find the $\epsilon$ value of apo iNicSnFR12, which
in previous models was assumed to be 0, but I don't
think this can be so easily simplified.

In [5]:
colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628','#f781bf']

p = bokeh.plotting.figure(
    frame_width=325,
    frame_height=250,
    y_axis_label="OD",
    x_axis_label="Wavelength",
    title='Absorption spectra for x uM apo iNicSnFR12'
    )



p.line(df['Wavel.'], df['0'], color=colors[0], legend_label='0')
p.line(df['Wavel.'], df['10'], color=colors[1], legend_label='10')
p.line(df['Wavel.'], df['50'], color=colors[2], legend_label='50')

p.legend.location = 'top_right'

bokeh.io.show(p)

Indeed, we see that at 496 nm there is an increase, even though
only slightly, in the OD at the 496 nm peak observed in the
bound iNicSnFR12. Using the data here we can estimate the $\epsilon$
of the unbound protein, $\epsilon_{u}$, to a reasonable degree.

In [7]:
max_A_norm_un = []

start = 73
data_un = [np.array(df['0'].values), np.array(df['10'].values), np.array(df['50'].values)]
for mean in data_un:
    # (OD of max absorbance, std of max absorbance, nm the peak it at)
    max_val = np.max(mean[start:50+start])
    max_index = list(mean[start:50+start]).index(max_val)

    max_A_norm_un.append(mean[73])

496
496
496


Here I gather the data points. I am collecting the absorbance values
at 496 nm from each dataset.

In [8]:
x = np.array([0, 10, 50])*.294
y = np.array(max_A_norm_mad)

res = linregress(x, y)

p = bokeh.plotting.figure(
    frame_width=325,
    frame_height=250,
    y_axis_label="Abosrbance at 496 nm",
    x_axis_label="unbound iNicSnFR12 (uM)",
    
    )
y_regress = res.slope * x + res.intercept
p.circle(x, y, legend_label="Data")
p.line(x=x, y=y_regress, color='red', legend_label="Regression", name="regression")
p.legend.location = 'top_left'
bokeh.io.show(p)

print('epsilon = {} /(M*cm)'.format(round(res.slope*10**6,1)))

ep_u = round(res.slope*10**6,1)

epsilon = 738.6 /(M*cm)


Using a linear regression fit we see that 
$\epsilon_u = 738.1$ /(M * cm). This is
two degrees of magnitude less than the expected
epsilon of the bound protein, $\epsilon_b$ , but still *not* equal to 0.

In [9]:
fname1 = 'experiment_vals_11_6.csv'
fname2 = 'experiment_conditions_11_6.csv'

if g_colab:
    df = pd.read_csv(io.BytesIO(uploaded[fname1]))
    df2 = pd.read_csv(io.BytesIO(uploaded[fname2]))
else:
    df = pd.read_csv(fname1)
    df2 = pd.read_csv('experiment_conditions_11_6.csv')

df.head()

,wavelength,0,1,2,3,4,5,6,7,8,...,44,45,46,47,48,49,50,51,52,53
0,350,0.2031,0.1224,0.1999,0.1221,0.2021,0.1226,0.2049,0.1223,0.2021,...,0.2157,0.1223,0.2159,0.1242,0.2181,0.1230,0.2163,0.1226,0.2160,0.1241
1,351,0.2056,0.1211,0.2025,0.1207,0.2046,0.1219,0.2079,0.1215,0.2056,...,0.2180,0.1208,0.2188,0.1223,0.2214,0.1219,0.2197,0.1215,0.2192,0.1228
2,352,0.2093,0.1197,0.2060,0.1208,0.2079,0.1214,0.2110,0.1204,0.2090,...,0.2215,0.1194,0.2217,0.1216,0.2242,0.1214,0.2230,0.1201,0.2226,0.1226
3,353,0.2117,0.1185,0.2081,0.1189,0.2108,0.1191,0.2132,0.1191,0.2107,...,0.2251,0.1195,0.2253,0.1206,0.2273,0.1192,0.2258,0.1190,0.2263,0.1202
4,354,0.2152,0.1176,0.2118,0.1176,0.2138,0.1182,0.2171,0.1177,0.2149,...,0.2286,0.1177,0.2286,0.1195,0.2305,0.1181,0.2291,0.1180,0.2285,0.1183


In [18]:
df2.head()

,experiment,[nicotine] uM,[iNicSnFR12] uM,path length cm
0,1,0.000000,50.000000,0.294118
1,2,20.568071,48.971596,0.300294
2,3,40.307102,47.984645,0.306471
3,4,80.882353,45.955882,0.320000
4,5,119.718310,44.014085,0.334118


Above are the data from the 9 experiments as well as the 
experimental conditions for each respective experiment.

In [11]:
data_minus_blank = []

for i in range(0,54,2):
    if (i)%6 == 0:
        data_minus_blank.append({})
    for j in range(3):
        data_minus_blank[-1][str(i+1) + ' ' + str(j+1)] = df[str(i)].values - df[str(i+1)].values

Here I subtract the blank data from each experimental dataset.

In [12]:
data_avg = []

for i in range(9):
    my_data = pd.DataFrame(data=data_minus_blank[i])
    mean = my_data.mean(axis=1).values
    std = my_data.std(axis=1).values
    data_avg.append((mean, std))

Here, since the experiments are run in triplicate, I compute
the average of each triplicate experiment from experiment 1 to experiment 9.

In [13]:
colors2 = ['#fcfbfd','#efedf5','#dadaeb','#bcbddc','#9e9ac8','#807dba','#6a51a3','#54278f','#3f007d']


p = bokeh.plotting.figure(
    frame_width=325,
    frame_height=250,
    y_axis_label="OD",
    x_axis_label="Wavelength",
    title='Absorbance spectra'
    )
for i,tup in enumerate(data_avg):
    mean, std = tup
    nic = round(df2['[nicotine] uM'].values[i],1)
    prot = round(df2['[iNicSnFR12] uM'].values[i],1)
    
    base = df['wavelength']

    p.line(base, mean, width=1, color=colors2[i], legend_label = str(i+1))

p.legend.location = 'top_right'
bokeh.io.show(p)

Above is the plotted average data from all 9 experiments.
This data has only been modified to subtract the blank
of the buffer+nicotine as well as computing the average.
No normalization has been performed.

In [14]:
path_length = np.array(df2['path length cm'].values)
maxes_norm = []
A_496 = []

conc_prot = np.array(df2['[iNicSnFR12] uM'].values)
conc_nic = np.array(df2['[nicotine] uM'].values)
conc_bound = []

#for i in range(len(conc_prot)):
    #conc_bound.append(get_snfr3(conc_[i], conc_nic[i]))
#print(conc_bound)
#conc_bound = [0,1.30617,2.36439,4.10122,5.37309,6.40863,7.25361,7.94657,8.5255]
#conc_unbound = conc_tot - conc_bound

start = 146
#print(data_avg[0][0])
for i,mean in enumerate(data_avg):
    mean = mean[0]
    val_496 = mean[start]
    
    #A_496.append(round(val_496 - ep_u*10**(-6) * conc_unbound[i]*path_length[i],4))
    A_496.append(round(val_496,4))

for val in A_496:
    print(val)
    
A_496 = np.array(A_496)

0.0131
0.0385
0.0605
0.0899
0.1147
0.1317
0.1414
0.1489
0.1537


Here I compute the absorbance at 496 nm. I did not subtract the contributions of the unbound protein because they do not contribute to the overall fit and I thought the data would be 'messier' that way.

What I mean by this is that the contributions of the unbound protein are estimated based on its assumed Kd. I wondered that if I used an assumed Kd while processing, it could be argued that the results were manipulated to match a particular Kd rather than letting the data speak for itself. The only points that differ greatly when I consider the unbound protein's absorbance are the first two points, which differ from .0131 -> .0027 and .0385 -> .027. Those values were computed by using the Kd of iNicSnFR12 to determine the steady state concentrations of bound and unbound sensor and then adjusting the absorbace accordingly by subtracting [unbound iNicSnFR12] * $\epsilon_u$ * path length. Otherwise, the points remain quite similar.

This is because as the solution becomes more saturated with nicotine, the contributions of the unbound protein are almost negligable. Even at lower concentrations of nicotine, the epsilon of the unbound protein is so low that
it does not create a large discrepancy. The original assumption of neglecting the absorbance of the unbound protein seems reasonable, and so I will continue with that assumption.

In [15]:
dA_dAmax = (A_496 - np.min(A_496))/np.max(A_496)

bound_prot = conc_prot * dA_dAmax

unbound_prot =conc_prot - bound_prot

Kds = conc_nic * unbound_prot / conc_prot

Kds

array([ 0.        , 17.16905301, 27.87667478, 40.46748823, 40.58115775,
       36.46194977, 33.05139883, 27.96471237, 23.86958035])

I compute the $k_D$ values. These were computed using the same formulas that Henry used in his Origin file.

In [16]:
p = bokeh.plotting.figure(
    frame_width=325,
    frame_height=250,
    y_axis_label="Abosrbance at 496 nm",
    x_axis_label="c_b * l (uM*cm)",
    
    )
x_lst = (path_length * bound_prot)
p.circle(x_lst, A_496)

bokeh.io.show(p)

These are the data plotted. The y axis is the absorbance at
496 nm and the x acis is the concentration of bound protein (c_b) multiplied by the path length. Each point represents an experiment 1-9.

In [17]:
x = x_lst
y = A_496

res = linregress(x, y)

p = bokeh.plotting.figure(
    frame_width=325,
    frame_height=250,
    y_axis_label="Abosrbance at 496 nm",
    x_axis_label="c_b * l (uM*cm)",
    
    )

y_regress = res.slope * x + res.intercept
p.circle(x, y, legend_label="Data")
p.line(x=x, y=y_regress, color='red', legend_label="Regression", name="regression")
p.legend.location = 'top_left'
bokeh.io.show(p)

print('epsilon = {} /(M*cm)'.format(round(res.slope*10**6,1)))

epsilon = 10451.6 /(M*cm)


With the entire data plotted, we get that $\epsilon_b=10,424$, which is
80% less than expected. 